In [96]:
import pandas as pd
import numpy as np

In [97]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [98]:
def mutate_data(data, train):
    data['IsCat'] = data['AnimalType'].map({'Cat':1, 'Dog':0})
    data['Gender'] = data['SexuponOutcome'].map({'Neutered Male':1, 'Spayed Female':2, 'Intact Male':3, 'Intact Female':4, 'Unknown':5, np.nan:0})
    
    # map age to weeks
    allAges = data.AgeuponOutcome
    age_in_days = []
    for age in allAges:
        try:
            n = int(age[0 : age.find(' ')])
            dur = age[age.find(' ') + 1]
            if (dur == 'd'):
                age_in_days.append(n)
            elif (dur == 'w'):
                age_in_days.append(n * 7)
            elif (dur == 'm'):
                age_in_days.append(n * 30)
            else:
                age_in_days.append(n * 365)
        except AttributeError:
            age_in_days.append(0)
    maxage = max(age_in_days)
    norm_age_in_days = [ float(i) / maxage for i in age_in_days]
    data['Age'] = norm_age_in_days
    
    # map breed to number
    breeds = data.Breed.unique()
    breed_map = {}
    for breed in breeds:
        breed_map[breed] = len(breed_map)
    data['Breed'] = data['Breed'].map(breed_map)
    
    if (train):
        data['Outcome'] = data['OutcomeType'].map({'Adoption':1, 'Died':2, 'Euthanasia':3, 'Return_to_owner':4, 'Transfer':5})
        data = data.drop('OutcomeSubtype', 1)
        data = data.drop('OutcomeType', 1)
        data = data.drop('AnimalID', 1)
        
    return data.drop(['SexuponOutcome', 'AnimalType', 'Name', 'DateTime', 'Color', 'AgeuponOutcome'], 1)

In [99]:
train_prep = mutate_data(train_data, True)
train_prep.to_csv('./train_prep.csv', index= False)

In [102]:
test_prep = mutate_data(test_data, False)
test_prep.to_csv('./test_prep.csv', index= False)